In [1]:
!pip install geemap >> /dev/null
!pip install pycrs >> /dev/null
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-testgee')

In [8]:
afg_central=geemap.shp_to_ee('/content/central_r.shp')
imgCol = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate('2018-01-01', '2018-12-31').filterBounds(afg_central).filter(ee.Filter.lt('CLOUD_COVER', 20))
img2018 = imgCol.reduce(ee.Reducer.percentile([20]))
img2018 = img2018.select(['SR_B2_p20', 'SR_B3_p20', 'SR_B4_p20', 'SR_B5_p20', 'SR_B6_p20']).clip(afg_central)
Map = geemap.Map(center = [33.9,67.7], zoom = 6)
vis_Para = {'min': 0, 'max': 30000, 'bands': ['SR_B4_p20', 'SR_B3_p20', 'SR_B2_p20']}
Map.addLayer(img2018, vis_Para, name="Exaple LANDSAT Image")
Map

Map(center=[33.9, 67.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [ ]:
lc_gt_train = geemap.shp_to_ee('/content/afg_gt_train.shp')
example_pnts = img2018.sampleRegions(lc_gt_train, scale=30)

In [11]:
my_classifier = ee.Classifier.smileRandomForest(numberOfTrees=200).train(example_pnts, 'lc_id')
afg_central_class = img2018.classify(my_classifier)
Map = geemap.Map(center = [33.9,67.7], zoom = 6)
vis_Para = {'min': 0, 'max': 5, 'palette':['grey', 'green', 'red', 'blue', 'yellow', 'purple']}
Map.addLayer(afg_central_class, vis_Para, name="Afg Central Region - LC")
Map


Map(center=[33.9, 67.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [12]:
lc_gt_val = geemap.shp_to_ee('/content/afg_gt_val.shp')
validation_pnts = img2018.sampleRegions(lc_gt_val, scale=30)
validation_class = validation_pnts.classify(my_classifier)
val_accuracy = validation_class.errorMatrix('lc_id', 'classification')
print(val_accuracy.accuracy().getInfo())

0.6219354838709678


In [22]:
#Improve accuracy by adding quarterly NDVI

def getNDVI(startY,endY, startM, endM):
  imgCol=ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterBounds(afg_central)
  imgCol=imgCol.filter(ee.Filter.calendarRange(startY,endY, 'year')).filter(ee.Filter.calendarRange(startM,endM, 'month'))
  imgCol=imgCol.filter(ee.Filter.lt('CLOUD_COVER',20))

  tempImg = imgCol.reduce(ee.Reducer.percentile([20])).clip(afg_central)

  ndvi = tempImg.expression('(NIR-RED)/(NIR+RED)',{'NIR':tempImg.select('SR_B5_p20'),'RED':tempImg.select('SR_B4_p20')})

  return ndvi

ndvi_q1 = getNDVI(2016, 2020, 1, 3).rename('ndvi_q1')
ndvi_q2 = getNDVI(2016, 2020, 4, 6).rename('ndvi_q2')
ndvi_q3 = getNDVI(2016, 2020, 7, 9).rename('ndvi_q3')
ndvi_q4 = getNDVI(2016, 2020, 10, 12).rename('ndvi_q4')

img_stack =ee.Image([img2018, ndvi_q1,ndvi_q2, ndvi_q3,ndvi_q4])

In [23]:
img_stack


In [24]:
example_pnts = img_stack.sampleRegions(lc_gt_train, scale=30)

In [25]:
my_classifier = ee.Classifier.smileRandomForest(numberOfTrees=200).train(example_pnts, 'lc_id')
afg_central_class = img_stack.classify(my_classifier)
Map = geemap.Map(center = [33.9,67.7], zoom = 6)
vis_Para = {'min': 0, 'max': 5, 'palette':['grey', 'green', 'red', 'blue', 'yellow', 'purple']}
Map.addLayer(afg_central_class, vis_Para, name="Afg Central Region - LC")
Map

Map(center=[33.9, 67.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [26]:
lc_gt_val = geemap.shp_to_ee('/content/afg_gt_val.shp')
validation_pnts = img_stack.sampleRegions(lc_gt_val, scale=30)
validation_class = validation_pnts.classify(my_classifier)
val_accuracy = validation_class.errorMatrix('lc_id', 'classification')
print(val_accuracy.accuracy().getInfo())

0.775483870967742
